# Tidal signatures search

In Gaia data, can we detect the signatures of tides by looking along the orbit to see excesses of stars of tidal shifts?

Below, I use both a cutout of 4 degrees from gaia using simple absolute cuts and also J+24's sample of bayesian probabilities and make plots of CMD and proper motion space along the orbit.

# Setuo

In [ ]:
using Arya, CairoMakie
using LilGuys
import CairoMakie: save
red = COLORS[6];
import TOML

In [ ]:
import LinearAlgebra: normalize, ×

In [ ]:
# selection parameters

pm_max = 10
θ0 = -39.63
dm_err = 0.05
n_sigma_pm = 6
fig_dir = "./figures/"

In [ ]:
obs_props = TOML.parsefile("/astro/dboyea/dwarfs/sculptor_obs_properties.toml")


In [ ]:
include("filter_utils.jl")

In [ ]:
filt_params = DensityParams(read_file("sculptor/simple.toml"))

In [ ]:
distance = obs_props["distance"]

## Loading data

In [ ]:
function load_gaia_file(filename, ra0=filt_params.ra, dec0=filt_params.dec, θ=θ0)
	all_stars = LilGuys.read_fits(filename)
	all_stars[:, :G] = all_stars.phot_g_mean_mag

	all_stars[:, :xi], all_stars[:, :eta] = lguys.to_tangent(all_stars.ra, all_stars.dec, ra0, dec0)

	all_stars[:, :xi_p], all_stars[:, :eta_p] = lguys.to_orbit_coords(all_stars.ra, all_stars.dec, ra0, dec0, θ)
	
    icrs = [lguys.ICRS(ra=all_stars.ra[i], dec=all_stars.dec[i], distance=distance, 
        pmra=all_stars.pmra[i], pmdec=all_stars.pmdec[i], radial_velocity=0.)
        for i in 1:size(all_stars, 1)]

    gsr = lguys.transform.(lguys.GSR, icrs)

    all_stars[:, :pmra_gsr] = getproperty.(gsr, :pmra)
    all_stars[:, :pmdec_gsr] = getproperty.(gsr, :pmdec)
    all_stars
end

In [ ]:
all_stars = let 
	all_stars = load_gaia_file("sculptor/gaia_4deg_cen.fits")
	n_sig_pm = 6

	filt_basic = ruwe_filter(all_stars, filt_params)
	filt_basic .&= parallax_filter(all_stars, filt_params)
	filt_basic .&= abs.(all_stars.pmra)  .< pm_max
	filt_basic .&= abs.(all_stars.pmdec) .< pm_max

	z_pm = bivariate_z.(all_stars.pmra, all_stars.pmdec, filt_params.pmra, filt_params.pmdec, all_stars.pmra_error, all_stars.pmdec_error, all_stars.pmra_pmdec_corr)

	
	all_stars[!, :filt_qual] = filt_basic
	all_stars[!, :filt_cmd] = cmd_filter(all_stars, filt_params)

	all_stars[!, :filt_pm] = z_pm .< n_sig_pm
	all_stars[!, :filt_all] = all_stars.filt_qual .& all_stars.filt_cmd .& all_stars.filt_pm

	
	all_stars
end

In [ ]:
j24 = load_gaia_file("../../data/j24_sculptor_all.fits")

j24[!, :filt_qual] = j24.F_BEST .== 1.0
j24[!, :filt_cmd] = j24.PSAT_CMD .> 0.2
j24[!, :filt_pm] = j24.PSAT_PM .> 0.2
j24[!, :filt_all] = j24.PSAT .> 0.2

j24

In [ ]:
orbit_props = TOML.parsefile("/astro/dboyea/sculptor/orbits/orbit1/1e6_V32_r5.4/orbital_properties.toml")
idx_f = orbit_props["idx_f"]

orbit = LilGuys.read_fits("/astro/dboyea/sculptor/orbits/orbit1/1e6_V32_r5.4/skyorbit.fits")
orbit[!, :xi], orbit[!, :eta] = LilGuys.to_tangent(orbit.ra, orbit.dec, orbit.ra[idx_f], orbit.dec[idx_f])

orbit = orbit[idx_f-10:idx_f+10, :]

## Plotting functions

In [ ]:
macro savefig(name, fig=nothing)
	if fig === nothing
		fig = esc(:fig)
	end
	
	return quote
		filename = joinpath(fig_dir, $name) * ".pdf"
		save(filename, $fig)
		@info "saved figure to $filename"
	end
end

In [ ]:
function cmd_axis(gs)
	return Axis(gs,
		xlabel = "Bp-Rp",
		ylabel = "G",
		yreversed=true,
		limits = (-0.2, 2, 15, 21),
		xgridvisible=false,
		ygridvisible=false,
	)
end

In [ ]:
function xieta_axis(gs)
	return Axis(gs,
		xlabel=L"$\xi$ / degrees",
		ylabel=L"$\eta$ / degrees",
		aspect=DataAspect(),
		xgridvisible=false, 
		ygridvisible=false,
        xreversed=true,
	)
end

In [ ]:
function pm_axis(gp; dpm=11, kwargs...)
	return Axis(gp, 
		xlabel=L"$\mu_{\alpha*}$ / mas\,yr$^{-1}$",
		ylabel=L"$\mu_\delta$ / mas\,yr$^{-1}$",
		aspect=DataAspect(),
		limits= dpm .* (-1, 1, -1, 1),
		xgridvisible=false,
		ygridvisible=false,
	)
end

# Plots of entire samples

In [ ]:
let
	fig = Figure()
	ax = xieta_axis(fig[1, 1])
    ax.title = "Gaia"
    
	ax.limits = (-4, 4, -4, 4)
	
	df = all_stars[all_stars.filt_cmd .& all_stars.filt_qual, :]
	scatter!(df.xi, df.eta, color=:black, alpha=1, markersize=3, label="quality+parallax+cmd")

	df = all_stars[all_stars.filt_cmd .& all_stars.filt_qual .& all_stars.filt_pm, :]
	scatter!(df.xi, df.eta, color=red, alpha=1, markersize=3, label="+pm")
    Legend(fig[1,2], ax)
    
	@savefig "scl_matched_filter"
	fig
end

In [ ]:
let
	fig = Figure()
	ax = xieta_axis(fig[1, 1])
	ax.limits = (-2, 2, -2, 2)
    ax.title = "J+24"
	
	df = j24[j24.filt_cmd .& j24.filt_qual, :]
	scatter!(df.xi, df.eta, color=:black, alpha=1, markersize=3, label="quality+parallax+cmd")

	df = j24[j24.filt_all, :]
	scatter!(df.xi, df.eta, color=red, alpha=1, markersize=3, label="PSAT")
    Legend(fig[1,2], ax)
    
	fig
end

In [ ]:
cmd_x = [filt_params.cmd_cut[1:2:end]; filt_params.cmd_cut[1]]
cmd_y = [filt_params.cmd_cut[2:2:end]; filt_params.cmd_cut[2]];

In [ ]:
let
	fig = Figure()

	ax = cmd_axis(fig[1, 1])

	filt = copy(all_stars.filt_qual)

	scatter!(all_stars.bp_rp[filt], all_stars.G[filt], markersize=1, alpha=0.3, color=:black, label="parallax + quality cuts")

	filt .&= all_stars.filt_cmd .* all_stars.filt_pm
	scatter!(all_stars.bp_rp[filt], all_stars.G[filt], markersize=2, label="+ CMD + PM cuts")
	#lines!(iso.bp_rp, iso.G .+ dm)
	poly!(cmd_x, cmd_y, color=:transparent, strokecolor=COLORS[1], strokewidth=2)

	axislegend(position=:lt, markersize=10)

	@savefig "cmd_polygon.pdf"
	fig
end

In [ ]:
let
	fig = Figure()

	ax = cmd_axis(fig[1, 1])
	ax.title = "J24"

	filt = copy(j24.filt_qual)

	scatter!(j24.bp_rp[filt], j24.G[filt], markersize=1, alpha=0.3, color=:black, label="parallax + quality cuts")

	filt .&= j24.filt_all
	scatter!(j24.bp_rp[filt], j24.G[filt], markersize=2, label="+ CMD + PM cuts")
	#lines!(iso.bp_rp, iso.G .+ dm)
	poly!(cmd_x, cmd_y, color=:transparent, strokecolor=COLORS[1], strokewidth=2)

	axislegend(position=:lt, markersize=10)

	@savefig "cmd_polygon.pdf"
	fig
end

In [ ]:
let
	fig = Figure()
    ax = pm_axis(fig[1, 1], dpm=11)
    
	scatter!(all_stars.pmra_gsr, all_stars.pmdec_gsr, markersize=1, alpha=0.2, color=:black)
	filt = all_stars.filt_pm 
	scatter!(all_stars.pmra_gsr[filt], all_stars.pmdec_gsr[filt], markersize=6)

	fig
end

In [ ]:
let
	fig = Figure()
    ax = pm_axis(fig[1, 1], dpm=11)
    
	scatter!(all_stars.pmra, all_stars.pmdec, markersize=1, alpha=0.2, color=:black)
	filt = all_stars.filt_pm 
	scatter!(all_stars.pmra[filt], all_stars.pmdec[filt], markersize=6)

	fig
end

In [ ]:
let
	fig = Figure()
    ax = pm_axis(fig[1, 1], dpm=11)
    
	scatter!(j24.pmra, j24.pmdec, markersize=2, alpha=0.2, color=:black)
	filt = j24.filt_pm 
	scatter!(j24.pmra[filt], j24.pmdec[filt], markersize=6)

	fig
end

# Cutouts

## Defining the cutouts

Here, we begin to take cutouts along the orbit (more specifically, along the proper motion vector in the GSR frame as this is model-independent)

In [ ]:
rs_test = LinRange(-4, 4, 10)

In [ ]:
orbit_vector = [sind(θ0), cosd(θ0)]
bg_vector = ([orbit_vector; 0] × [0, 0, 1])[1:2]

In [ ]:
r_circ_cut = 0.25

## Utility functions

In [ ]:
function select_region_circle(allstars, centre; radius=0.5)
	x_cen, y_cen = centre

	r = @. sqrt((allstars.xi - x_cen)^2 + (allstars.eta - y_cen)^2)

	filt = r .< radius

	return filt
end

In [ ]:
function select_region_rect(all_stars, low, high; radius=1, θ=0)

    x = @. cosd(θ) * all_stars.xi_p + sind(θ) * all_stars.eta_p
    y = @. sind(θ) * all_stars.xi_p - cosd(θ) * all_stars.eta_p
    
	filt = abs.(y) .<= radius

    filt .&= low .<= x .< high
    
	return filt
end

In [ ]:
function plot_cmd_members!(all_stars, filters; labels=nothing)    
    if labels === nothing
        labels = ["" for _ in filters]
    end

    styles = [(; color=:grey, markersize=3), 
        (; markersize=5), 
        (; color=COLORS[2], markersize=7),
        (; color=COLORS[3], markersize=10),
    ]

    for i in 1:length(filters)
        df = all_stars[filters[i], :]
        plot_cmd!(df, label=labels[i]; styles[i]...)
	end
    
	N = sum(filters[end])
    return N
end

In [ ]:
function plot_tangent_members!(all_stars, filters; labels=false)
    if labels === nothing
        labels = ["" for _ in filters]
    end

    styles = [(; color=:grey, markersize=3), 
        (; markersize=5), 
        (; color=COLORS[2], markersize=7),
        (; color=COLORS[3], markersize=10),
    ]

    scatter!(all_stars.xi[all_stars.filt_all], all_stars.eta[all_stars.filt_all]; markersize=1, color=:black)
    
    for i in 1:length(filters)
        df = all_stars[filters[i], :]
        scatter!(df.xi, df.eta, label=labels[i]; styles[i]...)
	end

    lines!(orbit.xi, orbit.eta)

end

In [ ]:
function plot_pm_members!(all_stars, filters; labels=false)
    if labels === nothing
        labels = ["" for _ in filters]
    end

    styles = [(; color=:grey, markersize=3), 
        (; markersize=5), 
        (; color=COLORS[2], markersize=7),
        (; color=COLORS[3], markersize=10),
    ]

    for i in 1:length(filters)
        df = all_stars[filters[i], :]
        scatter!(df.pmra, df.pmdec, label=labels[i]; styles[i]...)
	end

    df = all_stars[filters[end], :]
	x, y, xe, ye = get_mean_pm(df)

	scatter!(x, y, #xerr=[pmra_cen_err], yerr=[pmdec_cen_err], 
	color=red, label="mean", markersize=15)
    errscatter!([x], [y], xerr=[xe], yerr=[ye], 
	color=red, label="mean")

end

In [ ]:
function get_mean_pm(df)
	pmra_cen = lguys.mean(df.pmra_gsr, (df.pmra_error .^ -2))
	pmra_cen_err = sqrt(1 / sum(df.pmra_error .^ -2))
	pmdec_cen = lguys.mean(df.pmdec_gsr, (df.pmdec_error .^ -2))
	pmdec_cen_err = sqrt(1 / sum(df.pmdec_error .^ -2))

	return pmra_cen, pmdec_cen, pmra_cen_err, pmdec_cen_err
end

In [ ]:
function plot_cmd!(df; kwargs...)
	scatter!(df.bp_rp, df.G, alpha=1; kwargs...)
end

## Plots for gaia

In [ ]:
Makie.spaces()

In [ ]:
function compare_regions(all_stars, region_filters, selection_filters; 
        region_labels = nothing, legend=true,
        r_max = 4
    )
    
    N = length(region_filters)
    
    fig = Figure(size=(800, 1200))
    labels = first.(selection_filters)
    for i in 1:N
        filt_r = region_filters[i]
        filts = [filt .& filt_r for (_ , filt) in selection_filters]
        
        ax = xieta_axis(fig[i, 1])
        ax.limits = r_max .* (-1, 1, -1, 1)
        plot_tangent_members!(all_stars, filts, labels=labels)
        
        ax_cmd = cmd_axis(fig[i, 2])
        Ns = plot_cmd_members!(all_stars, filts, labels=labels)
        text!(ax_cmd, 0.05, 0.95, text="$Ns", space=:relative, align=(:left, :top, ))
            
        if region_labels !== nothing
            ax_cmd.title = region_labels[i]
        end
        
        ax_pm = pm_axis(fig[i, 3])
        plot_pm_members!(all_stars, filts, labels=labels)

        # hide decorations
        if i < N
            hidexdecorations!(ax, grid=false, ticks=false, minorticks=false)
            hidexdecorations!(ax_pm, grid=false, ticks=false, minorticks=false)
            hidexdecorations!(ax_cmd, grid=false, ticks=false, minorticks=false)
        end
    end

    if legend && (labels !== nothing)
        Legend(fig[N+1, 2], fig.content[1], tellwidth=false)
    end
    fig
end


In [ ]:
Nb = length(rs_test) - 1

In [ ]:
# regions_along1 = [select_region(all_stars, r * orbit_vector, radius=r_circ_cut) for r in rs_test]
# regions_along2 = [select_region(all_stars, -r * orbit_vector, radius=r_circ_cut) for r in rs_test]
# regions_perp1 = [select_region(all_stars, r * bg_vector, radius=r_circ_cut) for r in rs_test]
# regions_perp2 = [select_region(all_stars, r * bg_vector, radius=r_circ_cut) for r in rs_test]

r_bin_mid = midpoints(rs_test)
regions_along = [select_region_rect(all_stars, rs_test[i], rs_test[i+1]) for i in 1:Nb]
regions_perp = [select_region_rect(all_stars, rs_test[i], rs_test[i+1], θ=90) for i in 1:Nb]

labels_along = ["$r ° along orbit" for r in r_bin_mid]
labels_perp = ["$r ° ⟂ orbit" for r in r_bin_mid]

In [ ]:
filts_all_stars = [
    "quality" => all_stars.filt_qual, 
    "cmd" => all_stars.filt_qual .& all_stars.filt_cmd]


## J24 regions

In [ ]:
filts_j24 = [
    "quality"=>j24.filt_qual, 
    "psat"=>j24.filt_all
]

filts_j24_2 = [
    "quality" => j24.filt_qual, 
    "+cmd" => j24.filt_cmd .& j24.filt_qual]

rs_j24 = rs_test[abs.(rs_test) .< 2]
Nb_j24 = length(rs_j24) - 1

regions_along_j24 = [select_region_rect(j24, rs_j24[i], rs_j24[i+1]) for i in 1:Nb_j24]
regions_perp_j24 = [select_region_rect(j24, rs_j24[i], rs_j24[i+1], θ=90) for i in 1:Nb_j24]

rs_m_j24 = midpoints(rs_j24)

labels_along_j24 = ["$r ° along orbit" for r in rs_m_j24]
labels_perp_j24  = ["$r ° ⟂ orbit" for r in rs_m_j24]

In [ ]:
compare_regions(j24, regions_along_j24, filts_j24,
    region_labels = labels_along_j24,
    r_max=2
)

In [ ]:
compare_regions(j24, regions_j24_perp1, filts_j24_2,
    labels=labels_j24_2,
    region_labels = labels_perp1,
    r_max=2
)

In [ ]:
compare_regions(j24, regions_j24_along1, filts_j24,
    labels=labels_j24,
    region_labels = labels_along1,
    r_max=2
)

## All stars

In [ ]:
compare_regions(all_stars, regions_along, filts_all_stars,
    region_labels = labels_along
)

In [ ]:
compare_regions(all_stars, regions_perp, filts_all_stars,
    region_labels = labels_perp,
)

In [ ]:
compare_regions(all_stars, regions_perp2, filts_all_stars,
    labels=labels_all_stars,
    region_labels = labels_perp2,
)

## All stars + PM

In [ ]:
filts_all_stars2 = [all_stars.filt_qual, all_stars.filt_qual .& all_stars.filt_cmd .& all_stars.filt_pm]
labels_all_stars2 = ["quality", "cmd+pm"]

In [ ]:
compare_regions(all_stars, regions_along1, filts_all_stars2,
    labels=labels_all_stars2,
    region_labels = labels_along1
)

# Binned properties along orbit

In [ ]:
using DataFrames

In [ ]:
function compute_pm_stats(all_stars::DataFrame, r_bins)
    # Define the radial bins
    Nb = length(r_bins) - 1

    # Initialize arrays to store the results
    pmra_means = Float64[]
    pmdec_means = Float64[]
    pmra_means_err = Float64[]
    pmdec_means_err = Float64[]
    pm_counts = Int[]

    # Iterate through the radial bins
    for i in 1:Nb
        # Select stars in the current bin
        filt = select_region_rect(all_stars, r_bins[i], r_bins[i+1])
        
        # Calculate the mean and errors for proper motions
        x, y, xe, ye = get_mean_pm(all_stars[filt, :])

        # Append the results to the respective arrays
        push!(pmra_means, x)
        push!(pmdec_means, y)
        push!(pmra_means_err, xe)
        push!(pmdec_means_err, ye)
        push!(pm_counts, sum(filt))
    end

    # Create a DataFrame with the computed statistics
    df = DataFrame(
        r_bin_start = r_bins[1:end-1],
        r_bin_end = r_bins[2:end],
        r = midpoints(r_bins),
        pmra_mean = pmra_means,
        pmdec_mean = pmdec_means,
        pmra_mean_err = pmra_means_err,
        pmdec_mean_err = pmdec_means_err,
        pm_count = pm_counts
    )

    return df
end

In [ ]:
r_bins = LinRange(-4, 4, 10)

In [ ]:
df_means = compute_pm_stats(all_stars[all_stars.filt_all, :], r_bins)

df_nopm = compute_pm_stats(all_stars[all_stars.filt_cmd .& all_stars.filt_qual, :], r_bins)

In [ ]:
r_m = midpoints(r_bins)

In [ ]:
let

    fig = Figure(size=(800, 400))
	ax = Axis(fig[1, 1],
		xlabel="distance along orbit / degrees", 
		ylabel=L"$\tilde\mu_{\alpha*}$ / mas\,yr$^{-1}$",
		#limits=(-1.1, 1.1, 0, 0.3)
	)

	errscatter!(df_means.r, df_means.pmra_mean, yerr=df_means.pmra_mean_err, label="pm filter")
	errscatter!(df_nopm.r, df_nopm.pmra_mean, yerr=df_nopm.pmra_mean_err, label="no pm filter")


    hlines!(
        median(all_stars.pmra_gsr[all_stars.filt_qual .& .! all_stars.filt_cmd .& .! all_stars.filt_pm]), 
        label="nonmember median",
        color=COLORS[2],
    )


    Axis(fig[1, 2],
		xlabel="distance along orbit / degrees", 
		ylabel=L"$\tilde\mu_{\delta}$ / mas\,yr$^{-1}$",
		#limits=(-1.1, 1.1, 0, 0.3)
	)

	errscatter!(df_means.r, df_means.pmdec_mean, yerr=df_means.pmdec_mean_err)
	errscatter!(df_nopm.r, df_nopm.pmdec_mean, yerr=df_nopm.pmdec_mean_err)
    hlines!(
        median(all_stars.pmdec_gsr[all_stars.filt_qual .& .! all_stars.filt_cmd .& .! all_stars.filt_pm]), 
        label="nonmember mean",
    color=COLORS[2],
    )
    

    Legend(fig[1, 3], ax)
    
    fig
end

In [ ]:
let

    fig = Figure(size=(800, 400))
	ax = Axis(fig[1, 1],
		xlabel="distance along orbit / degrees", 
		ylabel=L"$\tilde\mu_{\alpha*}$ / mas\,yr$^{-1}$",
		#limits=(-1.1, 1.1, 0, 0.3)
	)

	errscatter!(df_means.r, df_means.pmra_mean, yerr=df_means.pmra_mean_err, label="pm filter")



    Axis(fig[1, 2],
		xlabel="distance along orbit / degrees", 
		ylabel=L"$\tilde\mu_{\delta}$ / mas\,yr$^{-1}$",
		#limits=(-1.1, 1.1, 0, 0.3)
	)

	errscatter!(df_means.r, df_means.pmdec_mean, yerr=df_means.pmdec_mean_err)

    fig
end

In [ ]:
import StatsBase: median

In [ ]:
let

	fig, ax = FigAxis(
		xlabel="distance along orbit / degrees", 
		ylabel=L"$\tilde\mu_{\delta}$ / mas\,yr$^{-1}$",
		#limits=(-1.1, 1.1, 0, 0.3)
	)

	errscatter!(df_means.r, df_means.pmdec_mean, yerr=df_means.pmdec_mean_err)
	errscatter!(df_nopm.r, df_nopm.pmdec_mean, yerr=df_nopm.pmdec_mean_err)
    hlines!(
        median(all_stars.pmdec_gsr[all_stars.filt_qual .& .! all_stars.filt_cmd .& .! all_stars.filt_pm]), 
        label="nonmember mean",
    color=COLORS[2],
    )

    fig
	

end

In [ ]:
let
    fig = Figure()
    ax = xieta_axis(fig[1,1])

    filt = all_stars.filt_qual .& .! all_stars.filt_cmd .& .! all_stars.filt_pm

    scatter!(all_stars.xi[filt], all_stars.eta[filt], color=:black, markersize=1)

    fig
end

In [ ]:
let
    fig = Figure()
    ax = xieta_axis(fig[1,1])

    filt = j24.PSAT .< 0.2

    scatter!(j24.xi[filt], j24.eta[filt], color=:black, markersize=1)

    fig
end